# BrainTrust Classification Tutorial (Article Titles)

<a target="_blank" href="https://colab.research.google.com/github/braintrustdata/braintrust-examples/blob/main/classify/py/BrainTrust-Classify-Tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Welcome to [BrainTrust](https://www.braintrustdata.com/)! This is a quick tutorial on how to build and evaluate an AI app to classify news titles into categories.

Before starting, make sure that you have a BrainTrust account. If you do not, please [sign up](https://www.braintrustdata.com) first. After this tutorial, learn more by visiting [the docs](http://www.braintrustdata.com/docs).

In [ ]:
# Set your API keys here
%env OPENAI_API_KEY=<YOUR_OPENAI_KEY>
%env BRAINTRUST_API_KEY=<YOUR_BRAINTRUST_API_KEY>

First, we'll install some dependencies.

In [ ]:
%pip install -U braintrust openai datasets autoevals

Next, we'll import the [ag_news dataset](https://huggingface.co/datasets/ag_news) from Huggingface.

In [ ]:
from datasets import load_dataset

# Load dataset from Huggingface.
dataset = load_dataset("ag_news", split="train")

# Extract category names from the dataset and build a map from index to
# category name. We will use this to compare the expected categories to
# those produced by the model.
category_names = dataset.features['label'].names
category_map = dict([name for name in enumerate(category_names)])

# Shuffle and trim to 20 datapoints. Restructure our dataset
# slightly so that each item in the list contains an input
# being the title and the expected category index label.
trimmed_dataset = dataset.shuffle(seed=42)[:20]
articles = [{
    "input": trimmed_dataset["text"][i],
    "expected": category_map[trimmed_dataset["label"][i]],
    } for i in range(len(trimmed_dataset["text"]))]


## Writing the initial prompts

Let's first write a prompt for categorizing a title for just one article. With BrainTrust, you can use any library you'd like -- Guidance, LangChain, or even just direct calls to an LLM.

The prompt provides the article's title to the model, and asks it to generate a category.

In [ ]:
# Here's the input and expected output for the first article in our dataset.
test_article = articles[0]
test_text = test_article["input"]
expected_text = test_article["expected"]

print("Article Title:", test_text)
print("Article Label:", expected_text)

In [ ]:
from openai import OpenAI

# The Braintrust Proxy will automatically cache results, which is convenient for development and evaluation where
# you often re-run notebooks, making small tweaks along the way.
#
# It is completely is optional (feel free to use any base_url here, or just remove it).
client = OpenAI(base_url="https://braintrustproxy.com/v1")

MODEL = "gpt-3.5-turbo"
SEED = 123

def classify_article(input):
    messages = [
        {
            "role": "system",
            "content": """You are an editor in a newspaper who helps writers identify the right category for their news articles,
by reading the article's title. The category should be one of the following: World, Sports, Business or Sci-Tech. Reply with one word corresponding to the category."""},
        {
            "role": "user",
            "content": "Article title: {article_title} Category:".format(article_title=input)
        },
    ]
    result = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=10,
        seed=SEED,
    )
    category = result.choices[0].message.content
    return category


test_classify = classify_article(test_text)
print("Input:", test_text)
print("Classified as:", test_classify)
print("Score:", 1 if test_classify == expected_text else 0)

## Running across the dataset

Now that we have automated classifying titles, we can test the full set of articles using Braintrust's eval function which parallelizes all of our generations and scoring for us.

In [ ]:
from braintrust import Eval
from autoevals import Levenshtein

await Eval(
    "classify-article-titles",
    data=articles,
    task=classify_article,
    scores=[Levenshtein]
)

## Pause and analyze the results in BrainTrust!

The cell above will print a link to the BrainTrust experiment. Click on it to investigate where we can improve our AI app.


Looking at our results table (in the screenshot below), we incorrectly output `sci-tech` instead of `sci/tech` which results in a failed eval test case. Let's fix it.

![issue.png](issue.png)

## Reproducing an example

First, let's see if we can reproduce this issue locally. We can test an article corresponding to the "Sci/Tech" category and reproduce the evaluation:

In [ ]:
sci_tech_article = [a for a in articles if "Galaxy Clusters" in a["input"]][0]
print(sci_tech_article["input"])
print(sci_tech_article["expected"])

out = classify_article(sci_tech_article["expected"])
print(out)

### Fixing the prompt

Have you spotted the issue? We have mispelled one of the categories in our prompt. The dataset's categories are "World, Sports, Business and *Sci/Tech*" - but we are using *Sci-Tech* in our prompt. Let's fix it:

In [ ]:
def classify_article_2(input):
    messages = [
        {
            "role": "system",
            "content": """You are an editor in a newspaper who helps writers identify the right category for their news articles,
by reading the article's title. The category should be one of the following: World, Sports, Business or Sci/Tech. Reply with one word corresponding to the category."""},
        {
            "role": "user",
            "content": "Article title: {input} Category:".format(input=input)
        },
    ]
    result = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=10,
        seed=SEED,
    )
    category = result.choices[0].message.content
    return category

result = classify_article_2(sci_tech_article["input"])

print(result)

### Evaluate the new prompt

The model classified the correct category `Sci/Tech` for this example. But, how do we know it works for the rest of the dataset? Let's run a new experiment to evaluate our new prompt using BrainTrust.

In [ ]:
await Eval(
    "classify-article-titles",
    data=articles,
    task=classify_article_2,
    scores=[Levenshtein]
)

## Conclusion

Click into the new experiment, and check it out! You should notice a few things:

![Compare](compare.png)

* BrainTrust will automatically compare the new experiment to your previous one.
* You should the eval scores increase and you can see which test cases improved.
* You can also filter the test cases that have a low score and work on improving the prompt for those

Now, you are on your journey of building reliable AI apps with BrainTrust.

Learn more on our docs @ [https://www.braintrustdata.com/docs](https://www.braintrustdata.com/docs).